##### Package imports

In [4]:
!jupyter nbextension enable --py widgetsnbextension
#interactive web map package
import folium 
from folium import features
import shapefile
from json import dumps
from folium import plugins

#Miscellanous
from IPython.core.display import display #display inline package
from ipywidgets import interact, interactive, fixed, FloatProgress
import ipywidgets as widgets
import pdb #debugging tool
import pandas as pd # pandas dataframe package
import pandas
from datetime import datetime #date conversion tool
from xlrd.xldate import xldate_as_tuple #xldate converter
from urllib2 import urlopen #get data from web tool
#from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image
import utm #lat long to utms
import zipfile #unzip tool
from IPython.display import clear_output
from scipy.signal import correlate #rainfall correlation

#numpy packages and tools
import numpy as np
from numpy import linspace
from numpy.fft import fft, ifft, fft2, ifft2, fftshift

#Spatial data packages
import rasterio #io raster data 
#from osgeo import gdal

#Plotly tools
import plotly.tools as tls
tls.set_credentials_file(username='NTPlotly', api_key='yViUde2xMRvWglgycSg4') #my plotly credentials (please dont use they cost me $$)
import plotly.plotly as py
#from plotly.graph_objs import *
import plotly.graph_objs as go
from plotly import tools
import cufflinks as cf
import matplotlib

#scipy packages
from scipy.misc import derivative #derivates of functions tool
from scipy.interpolate import UnivariateSpline #basic spline fitting tool
from scipy.signal import gaussian #gaussian filtering tool
from scipy.ndimage import filters #anotehr filtering tool
from scipy import stats #core scipy stats package
from scipy.integrate import simps #simpsons rule package for trapezoid calcualtions in area under curve calcs

#Basic math tools
from math import log
from math import factorial
import random

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
!pip install cufflinks

    100% |████████████████████████████████| 51kB 1.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for cufflinks ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/1d/79/29/bb076c9ab0bd7a932c33e7b6c7eb5d00d84001076f16549ec0
  Running setup.py bdist_wheel for colorlover ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/b8/b0/18/76d3f3088cc73950ed1aa44ea074b93ed37309fea57dec78dc
Successfully built cufflinks colorlover


####  Read data

Read mulitband image and dates csv into memory

In [5]:
def getOutputData(filename,pickleName,x,y):
    
    df = pd.read_csv(filename,header=0)
    
    df2 = pd.read_pickle(pickleName)
    
    #pdb.set_trace()
    
    modelledData = df.loc[(df["x"] == x) & (df["y"] == y)]
    
    dfV = df2.loc[(df2["x"] == x) & (df2["y"] == y)]
    
    dfV['dates'] = pandas.to_datetime(dfV['dates'])
    
    modelledData['sDate'] = pandas.to_datetime(modelledData['sDate'])
    
    modelledData['eDate'] = pandas.to_datetime(modelledData['eDate'])
    
    
    
    return dfV, modelledData   

##### Peak and trough plotting

In [6]:
def doPeakTroughPlots(df,modelledData):
       
    
    plot = []
    
    annotations = []
    
    #pdb.set_trace()
    
    modelledData = modelledData.set_index(['index'])
    
    for i in modelledData.index:
        
        if i == 0:
            
            a = go.Scatter(x=df['dates'], y=df['fittedSpline']-100.0,name='NPV Spline',mode = 'lines', line = dict(color = ('rgb(1, 1, 1)'),width = 1))
            
            b = go.Scatter(x=df['dates'], y=df['npv']-100.0,name='NPV',mode = 'markers',marker = dict(color = ('rgb(1, 1, 1)'),size = 2,symbol='circle'))
                 
            
            plot.append(a)
            
            plot.append(b)
        
        ind = modelledData.ix[i]
        
        #pdb.set_trace()
        
        minNpv = min(df['npv'])
        
        start = ind['sDate']
        
        end = ind['eDate']
        
        mask = (df['dates'] > start) & (df['dates'] <= end)
        
        sliced = df.loc[mask]
                        
        slicedLinStart = sliced.head(1)
        
        slicedLinEnd = sliced.tail(1)
        
        frames = [slicedLinStart,slicedLinEnd]
        
        result = pandas.concat(frames)
        
        
                                       
        st = str(start)
        
        st=st[:-9]
        
        ed = str(end)
        
        ed=ed[:-9]
        
        
        slicedEnd = sliced['fittedSpline'].iloc[len(sliced)-1]
        
        #slicedEnd = slicedEnd-100
        
        sliced['fittedSplineBase'] = slicedEnd
        
        
        #pdb.set_trace()
        
        #sln = go.Scatter(x=result['dates'],y=result['npv'], mode='line',line = dict(color = ('rgb(2, 2, 2)'),width = 5,),name=False)
                
         
        if i == 0:
            
            #pdb.set_trace()
            
            slnFitted = go.Scatter(x=result['dates'],y=result['fittedSpline']-100.0, mode='lines',name='Slope',line = dict(color = ('rgb(1, 1, 1)'),width = 4),showlegend=True)
                  
            pl = go.Scatter(x=sliced['dates'],y=sliced['fittedSpline']-100.0, fill = 'tozeroy',name = 'Area under the curve (small-integral)',fillcolor=('rgb(200, 200, 200)'),mode='none',line=dict(width=0.5,
              color='rgb(1, 1, 1)'),showlegend=True)
            
            pll = go.Scatter(x=sliced['dates'],y=sliced['fittedSplineBase']-100.0, fill = 'tozeroy',name = 'Area under the curve (large-integral)',fillcolor=('rgb(230, 230, 230)'),mode='none',line=dict(width=0.5,
              color='rgb(1, 1, 1)'),showlegend=True)
                       
            
            plot.append(pl)
            
            plot.append(slnFitted)
            
            plot.append(pll)
                                    
            annotations.append(dict(x=result['dates'].iloc[0],y=result['fittedSpline'].iloc[0]-100,xref='x',yref='y',text= 'Period ' + str(i),showarrow=True,arrowhead=7,ax=0,ay=-40))
        
        else:
            
            slnFitted = go.Scatter(x=result['dates'],y=result['fittedSpline']-100.0, mode='lines',name='Slope',line = dict(color = ('rgb(1, 1, 1)'),width = 4),showlegend=False)
      
                              
            pl = go.Scatter(x=sliced['dates'],y=sliced['fittedSpline']-100.0, fill = 'tozeroy',fillcolor=('rgb(200, 200, 200)'),mode='none',line=dict(width=0.5,
                  color='rgb(1, 1, 1)'),showlegend=False)
            
            pll = go.Scatter(x=sliced['dates'],y=sliced['fittedSplineBase']-100.0, fill = 'tozeroy',fillcolor=('rgb(230, 230, 230)'),mode='none',line=dict(width=0.5,
              color='rgb(1, 1, 1)'),showlegend=False)

            plot.append(pl)

            plot.append(slnFitted)
            
            plot.append(pll)
            
            annotations.append(dict(x=result['dates'].iloc[0],y=result['fittedSpline'].iloc[0]-100,xref='x',yref='y',text= 'Period ' + str(i),showarrow=True,arrowhead=7,ax=0,ay=-40))
                
   
    
    
    title = 'NPV time series'
            
    #layout = go.Layout(yaxis=dict(title='Non-Photosynthetic Vegetation Cover %'),xaxis=dict(title='Time'),font=dict(family='Arial, monospace', size=24, color = ('rgb(1, 1, 1)')))
    
    
    layout = go.Layout(title=title,yaxis=dict(title='Non-Photosynthetic Vegetation Cover %',range=[minNpv-100, 100]),xaxis=dict(title='Time'),font=dict(family='Arial, monospace', size=14, color = ('rgb(1, 1, 1)')),annotations = annotations)
      
    fig = go.Figure(data=plot, layout=layout)
    
    #url = py.iplot(fig,filename=filename)
    
    #pdb.set_trace()
    
    return fig

In [7]:
def doRescale(slopes,slopesF,auc,aucF,aucS):
    
    slopeRescaled = remap(slopes, min(slopes), max(slopes), 1, 100)

    aucRescaled = remap(auc, min(auc), max(auc), 1, 100)
        
    slopeRescaledFit = remap(slopesF, min(slopesF), max(slopesF), 1, 100)

    aucRescaledFit = remap(aucF, min(aucF), max(aucF), 1, 100)   
    
    aucSmallRescaledFit = remap(aucS, min(aucS), max(aucS), 1, 100)
    
    
    return slopeRescaled,aucRescaled,slopeRescaledFit,aucRescaledFit,aucSmallRescaledFit

In [8]:
def remap(x, oMin, oMax, nMin, nMax):

    #check reversed input range
    
    reverseInput = False
    
    #pdb.set_trace()
    
    oldMin = min( oMin, oMax )
    
    oldMax = max( oMin, oMax )
    
    if not oldMin == oMin:
    
        reverseInput = True

    #check reversed output range
    
    reverseOutput = False   
    
    newMin = min( nMin, nMax )
    
    newMax = max( nMin, nMax )
    
    if not newMin == nMin :
    
        reverseOutput = True

    portion = (x-oldMin)*(newMax-newMin)/(oldMax-oldMin)
    
    if reverseInput:
        
        portion = (oldMax-x)*(newMax-newMin)/(oldMax-oldMin)

    result = portion + newMin
    
    if reverseOutput:
        
        result = newMax - portion

    return result

##### Utilisation plotting

In [9]:
def doLocationPlots(df,modelledData):
     
  
    
    newDf = df.set_index(['dates'])
    
    sDateSliced = newDf.ix[modelledData['sDate']]
        
    eDateSliced = newDf.ix[modelledData['eDate']] 
   
        
    trace1=[]
    
    trace1a = go.Scatter(x=df['dates'],y=df['fittedSpline']-100.0,name='NPV Spline',mode = 'lines', line = dict(color = ('rgb(1, 1, 1)'),width = 2))
    
    trace1.append(trace1a)
    
    trace1b = go.Scatter(x=modelledData['sDate'],y=sDateSliced['fittedSpline']-100.0,name='Peaks',mode = 'markers', marker = dict(color = ('rgb(1, 1, 1)'),size = 13,symbol='circle'))
    
    trace1.append(trace1b)
    
    trace1c = go.Scatter(x=modelledData['eDate'],y=eDateSliced['fittedSpline']-100.0,name='Troughs',mode = 'markers', marker = dict(color = ('rgb(1, 1, 1)'),size = 13,symbol='triangle-up'))
   
    trace1.append(trace1c)
    
    trace1d = go.Scatter(x=df['dates'],y=df['npv']-100.0,name='NPV',mode = 'markers', marker = dict(color = ('rgb(1, 1, 1)'),size = 1,symbol='circle'))
    
    trace1.append(trace1d)
            
    layout = go.Layout(yaxis=dict(title='Non-Photosynthetic Vegetation Cover %'),xaxis=dict(title='Time'),font=dict(family='Arial, monospace', size=14, color = ('rgb(1, 1, 1)')))
      
    fig = go.Figure(data=trace1, layout=layout)
        
    #peakUrl = py.iplot(fig,filename=filename)   
    
    #pdb.set_trace()
    
    return fig,trace1,layout

In [10]:
def doXYPlot(x,y):
        
    dfV,modelledData = getOutputData("output.csv",'outputTimeSeriesPk.pkl',x,y)
    
    locFig,data,layout = doLocationPlots(dfV,modelledData)

    return locFig

A function to plot (heatmap) the temporal average area under the curve along with the selected x and y coordinate

In [11]:
#Function that is fired when the download button is clicked
def on_button_clickedPeakTrough(b):
    
    y = row.value
    
    x = col.value
            
    figPlot = doXYPlot(int(x),int(y))
    
    filename1 = "researchProject/peakTroughs1"
    
    peakPlot = py.iplot(figPlot, filename=filename1)
        
    display(peakPlot)
    

Text box's for x and y corrdinates

In [12]:
row = widgets.Text(description="y:(0-67)")
col = widgets.Text(description="x:(0-67)")
row.value = '5'
col.value = '5'
display(row)
display(col)

Bottons for the coordinate and plot displays

In [13]:
# Click download to import geotif image and csv date files to local directory
button = widgets.Button(description="Peak trough plot")
display(button)
button.on_click(on_button_clickedPeakTrough)

IOError: [Errno 2] No such file or directory: 'outputTimeSeriesPk.pkl'

In [29]:
#on_button_clickedPeakTrough('a')

Map of field site and image extent with high res imagery background

In [30]:
def worldImagery(location,site,df):

    m = folium.Map(location=location)

    url = 'http://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/MapServer/tile/{z}/{y}/{x}'

    m.add_tile_layer(tile_name="World_Imagery",tile_url=url, attr ="world imagery" )
    
    folium.Marker([location[0], location[1]], popup=site).add_to(m)
    
    coords = makeSceneBound(location)
    
    #pdb.set_trace()
    
    my_PolyLine=folium.PolyLine(locations=coords,weight=5)

    m.add_children(my_PolyLine)   
    
    sw = list(coords[3])
    
    ne = list(coords[1])
    
    m.fit_bounds([sw,ne])
       
    return m,coords

In [31]:
def getSiteLatLon(site,df):
        
    
    mask = df['sites'] == site
    
    lat = df['lat'][mask]
    
    lat = lat.iloc[0]
    
    longt = df['longt'][mask]
    
    longt = longt.iloc[0]
    
    return lat,longt
    

In [32]:
def makeSceneBound(location):
    
    utms = utm.from_latlon(location[0],location[1])
    
    north = utms[1]
    
    east = utms[0]
    
    trE = east + 1000
    
    trN = north + 1000
    
    trU = utm.to_latlon(trE,trN,utms[2],utms[3])
    
    tlE = east - 1000
    
    tlN = north + 1000
    
    tlU = utm.to_latlon(tlE,tlN,utms[2],utms[3])
    
    brE = east + 1000
    
    brN = north - 1000
    
    brU = utm.to_latlon(brE,brN,utms[2],utms[3])
    
    blE = east - 1000
    
    blN = north - 1000
    
    blU = utm.to_latlon(blE,blN,utms[2],utms[3])
    
    coords = [tlU,trU,brU,blU,tlU]
    
    return coords
    

In [33]:
#Enter siteId from map exactly as it is displayed ie. 'hmd20' or 'bsp14a'
text = widgets.Text(description="Site Id from map")
display(text)

In [36]:
#Function that is fired when the download button is clicked
def on_button_clickedPlotC(b):
    
    #site = text.value
    
    site = 'nhv03'
    
    
    df = pd.read_csv('siteDetails.csv')
    
    lat,longt = getSiteLatLon(site,df)
    location=[lat,longt]
    m,coords = worldImagery(location,site,df)
            
    display(m)
    
    return m,coords
    

In [39]:
#on_button_clickedPlotC('a')

In [38]:
# Click download to import geotif image and csv date files to local directory
button = widgets.Button(description="Site map")
display(button)
button.on_click(on_button_clickedPlotC)

#### 3.2.2. Utilisation periods

In [40]:
def doUtilisationPlots():
    
    y = row.value
    
    x = col.value     
    
    dfV,modelledData = getOutputData("output.csv",'outputTimeSeriesPk.pkl',int(x),int(y))
    
    fig = doPeakTroughPlots(dfV,modelledData)
           
    return fig     

In [41]:
#Function that is fired when the download button is clicked
def on_button_clickedPlotD(b):   
    
    figPlot = doUtilisationPlots()
    
    filename1 = "researchProject/utilisation"
    
    utiliPlot = py.iplot(figPlot, filename=filename1)
        
    display(utiliPlot)

In [42]:
# Click download to import geotif image and csv date files to local directory
button = widgets.Button(description="Site Utilisation plot")
display(button)
button.on_click(on_button_clickedPlotD)

In [56]:
def doUtilisationImage(period,x,y):  
    
    #pdb.set_trace()
    
    print str(period)
           
    data = pd.read_csv('output.csv')
    
    sliced = data.loc[data['index'] == period]
    
    #needs a window to account for when the chosen x y is null.
    
    tlx = x-1
    
    tly = y-1
    
    tmx = x
    
    tmy = y-1
    
    trx = x+1
    
    tRy = y-1
    
    clx = x -1
    
    cly = y 
    
    cx = x
    
    cy = y
    
    crx = x+1
    
    cry = y
    
    blx = x-1
    
    bly = y +1
    
    bmx = x
    
    bmy = y+1
    
    brx = x + 1
    
    bry = y + 1
    
    #pdb.set_trace()
    
    xWindow = [tlx,tmx,trx,clx,cx,crx,blx,bmx,brx]
    
    yWindow = [tly,tmy,tRy,cly,cy,cry,bly,bmy,bry]
    
    wind = pd.DataFrame()
    
    for ii in range(len(xWindow)):
        
        xSlice = sliced.loc[sliced['x']==xWindow[ii]]
        
        ySlice = xSlice.loc[xSlice['y']==yWindow[ii]]
        
        wind = pd.concat([wind,ySlice])  
    
    if wind.empty == True:
        
        raise Exception("no data")
    
    else:        
        
        
        start = wind['sDate'].iloc[0] 

        #end = wind['eDate'].iloc[0] 

        aucFBig = np.zeros((67,67))

        aucFSmall = np.zeros((67,67))

        slope = np.zeros((67,67))

        for i in range(67):

            #print i

            #i = 14

            for j in range(67):

                #print j

                #j = 23

                part1 = data.loc[data['x']==i] # slice data on row i

                part2 = part1.loc[part1['y']==j] # slice data on row i and col j

                #pdb.set_trace()          



                test = sum(part2['npv'])

                if test == 0:

                    aucFBig[j,i] = np.nan

                    aucFSmall[j,i] = np.nan

                    slope[j,i] = np.nan

                else:

                    closest = nearestDate(pd.to_datetime(start), pd.to_datetime(part2['sDate']))

                    mask = pd.to_datetime(part2['sDate']) == closest

                    match = part2[mask]

                    aucFBig[j,i] = match['aucFUSc']

                    aucFSmall[j,i] = match['aucSUSc']

                    slope[j,i] = match['slopeFUsc']

        #pdb.set_trace()
        
        aucFBig[aucFBig<=0]=np.nan

        aucFSmall[aucFSmall<=0]=np.nan

        slope[slope==0]=np.nan

        rescaledBig = remap(aucFBig, np.nanmin(aucFBig), np.nanmax(aucFBig), 0, 100)

        rescaledSmall = remap(aucFSmall, np.nanmin(aucFSmall), np.nanmax(aucFSmall), 0, 100)

        rescaledSlope  = remap(slope, np.nanmin(slope), np.nanmax(slope), 0, 100)

        return  rescaledBig, rescaledSmall, rescaledSlope

In [44]:
#simple function to find the closest date in a list of dates, i.e. find the closest image date to a monthly rainfall date
def nearestDate(base, dates):
    
    #pdb.set_trace()
        
    nearness = { abs(base - date) : date for date in dates }
    
    return nearness[min(nearness.keys())]

In [54]:
def doColourMapping(image,cType,opac,period):
    
    #pdb.set_trace()
    
    newImage = image.astype(np.float32)
    
    src = rasterio.open('chip.tif')
    
    meta = src.meta    
    
    if cType == 'single':
           
        with rasterio.open('single' + str(period) + '.tif', 'w', **meta) as dst:

            dst.write_band(1,newImage)

            clrTbl = {}

            table = []

            for i in range(200):

                if i < 100:

                    red = 165 + i + 1

                    green = i + 2

                    blue = 38 + i + 1

                elif i >100:

                    red = 224 - i + 1

                    green = 243 - i + 2

                    blue = 248 - i + 3

                colour = (red,green,blue,opac) 

                clrTbl[i] = colour

            #pdb.set_trace()
            
            dst.write_colormap(1,clrTbl)
   
    

In [46]:
def doHeatmap(array,x,y,title):
           
    data = np.flipud(array) #flip array prior to convert to a list, this is needed for when heatmap plots rows and columns    
    
    #pdb.set_trace()
   
    df4 = pandas.DataFrame(data)
       
    #colorscale=[[0.0, 'rgb(165,0,38)'], [0.1111111111111111, 'rgb(215,48,39)'], [0.2222222222222222, 'rgb(244,109,67)'], [0.3333333333333333, 'rgb(253,174,97)'], [0.4444444444444444, 'rgb(254,224,144)'], [0.5555555555555556, 'rgb(224,243,248)'], [0.6666666666666666, 'rgb(171,217,233)'], [0.7777777777777778, 'rgb(116,173,209)'], [0.8888888888888888, 'rgb(69,117,180)'], [1.0, 'rgb(49,54,149)']]
    
    colorscale=[[0.0, 'rgb(49,54,149)'], [0.1111111111111111, 'rgb(69,117,180)'], [0.2222222222222222, 'rgb(116,173,209)'], [0.3333333333333333, 'rgb(171,217,233)'], [0.4444444444444444, 'rgb(254,224,144)'], [0.5555555555555556, 'rgb(224,243,248)'], [0.6666666666666666, 'rgb(165,0,38)'], [0.7777777777777778, 'rgb(215,48,39)'], [0.8888888888888888, 'rgb(244,109,67)'], [1.0, 'rgb(253,174,97)']]
    
    
    
    data = [go.Heatmap(z=df4.values.tolist(), colorscale=colorscale),go.Scatter(x=x,y=y,mode = 'markers',marker = dict(size = 10,symbol = 'cross',color = 'rgba(0, 0, 0)'), line = dict(width = 2,color = 'rgb(0, 0, 0)'))]

    layout = go.Layout(title = title,width=600,height=600,xaxis= dict(title= 'x'),yaxis= dict(title= 'y'))
    
    fig = go.Figure(data=data,layout=layout)
               
    return fig

In [47]:
def doHeatmap2(array,x,y,title):
           
    data = np.flipud(array) #flip array prior to convert to a list, this is needed for when heatmap plots rows and columns    
         
    df4 = pandas.DataFrame(data)
             
    colorscale=[[0.0, 'rgb(255,0,0)'], [0.1111111111111111, 'rgb(204,0,0)'], [0.2222222222222222, 'rgb(153,0,0)'], [0.3333333333333333, 'rgb(102,0,0)'], [0.4444444444444444, 'rgb(0,53,0)'], [0.5555555555555556, 'rgb(0,0,0)'], [0.6666666666666666, 'rgb(0,255,0)'], [0.7777777777777778, 'rgb(0,204,0)'], [0.8888888888888888, 'rgb(0,153,0)'], [1.0, 'rgb(0,102,0)']]
      
    #colorscale=[[0.0, 'rgb(0,0,255)'], [0.1111111111111111, 'rgb(0,0,204)'], [0.2222222222222222, 'rgb(0,0,153)'], [0.3333333333333333, 'rgb(0,0,102)'], [0.4444444444444444, 'rgb(0,53,0)'], [0.5555555555555556, 'rgb(0,0,0)'], [0.6666666666666666, 'rgb(0,255,0)'], [0.7777777777777778, 'rgb(0,204,0)'], [0.8888888888888888, 'rgb(0,153,0)'], [1.0, 'rgb(0,102,0)']]
        
        
    data = [go.Heatmap(z=df4.values.tolist(),zmin=0, zmax=200, colorscale=colorscale),go.Scatter(x=x,y=y,mode = 'markers',marker = dict(size = 10,symbol = 'cross',color = 'rgba(0, 255, 0)'), line = dict(width = 2,color = 'rgb(0, 0, 0)'))]

    layout = go.Layout(title = title,width=600,height=600,xaxis= dict(title= 'x'),yaxis= dict(title= 'y'))
    
    fig = go.Figure(data=data,layout=layout)
              
                   
    return fig

In [48]:
def doHeatMapImages(period1,period2,x,y):
    
    #pdb.set_trace()
    
    rescaledBig1, rescaledSmall1, rescaledSlope1 = doUtilisationImage(period1,x,y)
    
    rescaledBig2, rescaledSmall2, rescaledSlope2 = doUtilisationImage(period2,x,y)
    
    
    deltaBig1 = rescaledBig1
    
    deltaBig2 = rescaledBig2    
   
    change = (deltaBig2/deltaBig1)-1
    
    bigPercentChange = change*100
    
    mask = bigPercentChange >= 100
    
    bigPercentChange[mask]=100
    
    mask = bigPercentChange <= -100
    
    bigPercentChange[mask]= -100
    
    bigPercentChange = bigPercentChange +100   
    
    
    #pdb.set_trace()    
    
    deltaSmall1 = rescaledSmall1
    
    deltaSmall2 = rescaledSmall2   
   
    change = (deltaSmall2/deltaSmall1)-1
    
    smallPercentChange = change*100  
    
    mask = smallPercentChange >= 100
    
    smallPercentChange[mask]=100
    
    mask = smallPercentChange <= -100
    
    smallPercentChange[mask]= -100
    
    smallPercentChange = smallPercentChange +100
    
            
    deltaSlope1 = rescaledSlope1
    
    deltaSlope2 = rescaledSlope2   
   
    change = (deltaSlope2/deltaSlope1)-1
    
    slopePercentChange = change*100 
    
    mask = slopePercentChange >= 100
    
    slopePercentChange[mask]=100
    
    mask = slopePercentChange <= -100
    
    slopePercentChange[mask]= -100
    
    slopePercentChange = slopePercentChange +100  
       
      
    
    return rescaledBig1, rescaledSmall1, rescaledSlope1, rescaledBig2, rescaledSmall2, rescaledSlope2, bigPercentChange, smallPercentChange, slopePercentChange 
    
    
    

In [49]:
def on_button_clickedPlotE(b):
    
    # First period
 
    #period1 = temp5.value -1
    
    period1 = temp5.value 
    
    #period2 = temp6.value -1
    
    period2 = temp6.value
   
    x = int(row.value)
    
    y = int(col.value)
    
    #pdb.set_trace()
    
    rescaledBig1, rescaledSmall1, rescaledSlope1, rescaledBig2, rescaledSmall2, rescaledSlope2, bigPercentChange, smallPercentChange, slopePercentChange = doHeatMapImages(period1,period2,x,y) 
        
    meth = temp4.value   
    
    if meth == 'b':
        
        #plot = doHeatmap(rescaledBig1,x,y,"Area under curve - large integral - Period "+str(period1+1))
        
        plot = doHeatmap(rescaledBig1,x,y,"Area under curve - large integral - Period "+str(period1))
        
    elif meth == 's':
        
        #plot = doHeatmap(rescaledSmall1,x,y,"Area under curve - small integral- Period "+str(period1+1))
        
        plot = doHeatmap(rescaledSmall1,x,y,"Area under curve - small integral- Period "+str(period1))
        
    elif meth == 'r':
        
        plot = doHeatmap(rescaledSlope1,x,y,"Rate of decline - Period "+str(period1))

    filename1 = "researchProject/heatP1"

    heatp1 = py.iplot(plot, filename=filename1)
       
    display(heatp1)    
    
    # second period
        
    if meth == 'b':
        
        #plot = doHeatmap(rescaledBig2,x,y,"Area under curve - large integral - Period "+str(period2+1))
        
        plot = doHeatmap(rescaledBig2,x,y,"Area under curve - large integral - Period "+str(period2))
        
    elif meth == 's':
        
        #plot = doHeatmap(rescaledSmall2,x,y,"Area under curve - small integral- Period "+str(period2+1))
        
        plot = doHeatmap(rescaledSmall2,x,y,"Area under curve - small integral- Period "+str(period2))
        
    elif meth == 'r':
        
        plot = doHeatmap(rescaledSlope2,x,y,"Rate of decline - Period "+str(period2))

    filename2 = "researchProject/heatP2"

    heatp2 = py.iplot(plot, filename=filename2)
    
    display(heatp2)
        
    # difference 
    
    if meth == 'b':
        
        plot = doHeatmap2(bigPercentChange,x,y,"Area under curve - large integral - Difference")
        
    elif meth == 's':
        
        plot = doHeatmap2(smallPercentChange,x,y,"Area under curve - small integral- Difference")
        
    elif meth == 'r':
        
        plot = doHeatmap2(slopePercentChange,x,y,"Rate of decline - Difference")

    filename3 = "researchProject/heatdif"
    
    heatdif = py.iplot(plot, filename=filename3)

    display(heatdif)   
    
    

In [50]:
temp4 = widgets.Dropdown(options ={'Rate of decline':'r','Area under curve - large integral':'b','Area under curve - small integral':'s'}) #

display(temp4)

In [52]:
utilData = pd.read_csv('output.csv')

mask = utilData['auc']>0

aa = utilData[mask]

maxs = aa['index'].max()

newDict = {}

for i in range(maxs):
    
    p = 'Period ' + str(i)
    
    newDict.update({p: i})

temp5 = widgets.Dropdown(options =newDict,description='1st period',button_style='success')

temp6 = widgets.Dropdown(options =newDict,description= '2nd period',button_style='info')

display(temp5)

display(temp6)

NOTE: I need to work through this a bit more, the differences between the plotted auc in the utilisation plot and the image heat maps is due to the rescaling that i do., also I think a better method of determining the date to use as the period is needed, at present this is the date or period selected from the utlisation plot of the centre coord and the rest of the pixels are matched closest to that date. I think a better method would be to somehow calculate the average  of all the pixels, not sure what this means???

In [57]:
button = widgets.Button(description="Plot Utilisation")
display(button)
button.on_click(on_button_clickedPlotE)

1
9


In [58]:
def writeData(data,filename):

    with rasterio.open('chip.tif') as src:

        profile = src.profile

        profile.update(dtype=rasterio.uint8, count=1, compress='lzw')

    with rasterio.open(filename, 'w', **profile) as dst:
    
        dst.write(data.astype(rasterio.uint8), 1)   
    

In [60]:
def imageOverlay():
    
    site = text.value
    
    period1 = temp5.value
    
    period2 = temp6.value

    x = int(row.value)
    
    y = int(col.value)
    
    df = pd.read_csv('siteDetails.csv')
    
    lat,longt = getSiteLatLon(site,df)
    
        
    location=[lat,longt]
    
    m = folium.Map(location=location,zoom_start=15)

    url = 'http://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/MapServer/tile/{z}/{y}/{x}'

    m.add_tile_layer(tile_name="World_Imagery",tile_url=url, attr ="world imagery" )
    
    folium.Marker([location[0], location[1]], popup=site).add_to(m)
    
    coords = makeSceneBound(location)
    
    #pdb.set_trace()
        
    min_lat = coords[0][0]
    
    max_lat = coords[2][0]
    
    min_lon = coords[0][1]
    
    max_lon = coords[2][1]        
    
    
    rescaledBig1, rescaledSmall1, rescaledSlope1, rescaledBig2, rescaledSmall2, rescaledSlope2, bigPercentChange, smallPercentChange, slopePercentChange = doHeatMapImages(period1,period2,x,y)
    
    #pdb.set_trace()        
    
    outputs1 = [rescaledBig1, rescaledSmall1, rescaledSlope1, rescaledBig2, rescaledSmall2, rescaledSlope2]
    
    names1 = ['AUC-large-P1', 'AUC-small-P1', 'Slope-P1', 'AUC-large-P2', 'AUC-small-P2', 'Slope-P2']
    
    outputs2 = [bigPercentChange, smallPercentChange, slopePercentChange]
    
    names2 = ['AUC-large-difference', 'AUC-small-difference', 'Slope-difference']
     
    #pdb.set_trace()
    for i in range(len(outputs1)):
        
        #pdb.set_trace()
        
        #cm = matplotlib.cm.get_cmap('terrain')
        
        cm = matplotlib.cm.get_cmap('Greys')

        colored = outputs1[i].astype(int)
        
        #normed_data = (colored - np.nanmin(colored)) / (np.nanmax(colored) - np.nanmin(colored))

        coloredData = cm(colored)
                                                        
        #aa = plugins.ImageOverlay(coloredData,bounds =[[min_lat, min_lon], [max_lat, max_lon]])
        
        matplotlib.pyplot.imsave('test.png', coloredData)
        
        aa = plugins.ImageOverlay(image=open('test.png'),opacity=0.9,bounds =[[min_lat, min_lon], [max_lat, max_lon]])
        
    
        aa.layer_name = names1[i]
        
        m.add_children(aa)
    
        #writeData(outputs1[i],'data/'+ names1[i] + '.tif')
     
    
        
    for i in range(len(outputs2)):
        
        if i == 2:
            
            cm = matplotlib.cm.get_cmap('RdBu')
            #cm = matplotlib.cm.get_cmap('Greys')
            
        else:
                    
            cm = matplotlib.cm.get_cmap('seismic')
            
            #cm = matplotlib.cm.get_cmap('Greys')

        colored = outputs2[i].astype(int)

        coloredData = cm(colored)
        
        matplotlib.pyplot.imsave('test.png', coloredData)
        
        aa = plugins.ImageOverlay(image=open('test.png'),opacity=0.5,bounds =[[min_lat, min_lon], [max_lat, max_lon]])
              
        #aa = plugins.ImageOverlay(coloredData, opacity=0.7,bounds =[[min_lat, min_lon], [max_lat, max_lon]])
    
        aa.layer_name = names2[i]
        
        m.add_children(aa)
    
    folium.LayerControl().add_to(m) 
    
    folium.LatLngPopup().add_to(m)
           
    
    display(m)    
    
    

In [61]:
def onButtonClickImageOverlay(b):
        
    
    imageOverlay()
    

In [62]:
button = widgets.Button(description="Image Overlay")

display(button)

button.on_click(onButtonClickImageOverlay)

IndexError: single positional indexer is out-of-bounds

In [34]:
def onButtonClickZ(b):
    
    filename = dropDownImages.value

    src = rasterio.open('data/'+filename)

    lat = latitude.value
    
    lat = float(lat)

    longt= longitude.value
    
    longt = float(longt)
    
    #pdb.set_trace()

    utms = utm.from_latlon(lat,longt)

    north = utms[1]

    east = utms[0]

    vals = src.sample([(east, north)])

    aa = list(vals)

    pixelValue = aa[0][0]
    
    print pixelValue,north,east

    centreEast = (src.bounds.left + src.bounds.right) / 2.0

    centreNorth = (src.bounds.bottom + src.bounds.top) / 2.0

    diffEast = east - centreEast

    diffNorth = centreNorth - north

    cellX = round(diffEast/30)

    cellY = round(diffNorth/30)

    newY = 67/2+cellX

    newX = 67/2+cellY

    dfV,modelledData = getOutputData("data/output.csv",'data/outputTimeSeriesPk.pkl',int(newX),int(newY))

    fig = doPeakTroughPlots(dfV,modelledData)

    filename1 = "researchProject/utilisation"

    utiliPlot = py.iplot(fig, filename=filename1)

    display(utiliPlot)


In [35]:
dropDownImages = widgets.Dropdown(options ={'AUC-large-P1':'AUC-large-P1.tif','AUC-small-P1':'AUC-small-P1.tif','Slope-P1':'Slope-P1.tif','AUC-large-P2':'AUC-large-P2.tif','AUC-small-P2':'AUC-small-P2.tif','Slope-P2':'Slope-P2.tif','AUC-large-difference':'AUC-large-difference.tif','AUC-small-difference':'AUC-small-difference.tif','Slope-difference':'Slope-difference.tif'}) #

display(dropDownImages)

The installed widget Javascript is the wrong version.


In [36]:
latitude = widgets.Text(description="Latitude")
longitude = widgets.Text(description="Longitude")
display(latitude)
display(longitude)

The installed widget Javascript is the wrong version.
The installed widget Javascript is the wrong version.


In [37]:
button = widgets.Button(description="Get coord plot")

display(button)

button.on_click(onButtonClickZ)

The installed widget Javascript is the wrong version.


# Text for dashboard

### Step 1. Select field site

#### Step 5B. Enter coordinates from map to re-plot utilisation

### Step 3. Download image

### Step 4. Click to process imagery

### Step 5. Display field site & image extent 

### Step 2. Select row and column value to plot 

### Step 3. Generate peak  & trough plot

### Step 4 . Generate utilisation plot

### Step 9A. Peak detection plot

### Step 9B. Peak lag assessment

#### Step 5A. Select Periods to compare

#### Step 5B. Map Comparisons

#### Step 5B. Enter coordinates from map click to re - plot utilisation

### Step 5. Utilisation Comparisons